# Intel Geti REST API Tutorial: How to create a Project and upload a media item
This tutorial will guide you through the process of creating a project and uploading an image to the Intel Geti platform using the REST API. Although this tutorial is written in Python, you can use any programming language that supports REST API calls. Throughout this tutorial the main reference point for you will be the Intel Geti REST API documentation, you can access it by following `https://YOUR_GETI_HOST/docs/rest/` in your browser. The Swagger page available at this address will provide you with all the necessary information about the endpoints, request and response bodies.

## Prerequisites
Before we start, make sure you have the following:
1. A Python 3 interpreter installed on your machine. Any version will work, but the Python 3.8 or newer is recommended.
2. To run this tutorial in your local environment, you will need to install the `requests` library. As well as the `jupyter` package to run the Jupyter Notebook.
    You can do this by running the following command in your terminal:
    ```bash
    python3 -m pip install requests jupyter notebook
    ```
3. An Intel Geti Platform Host address. If you don't have a Geti account, you can create it here https://geti.intel.com/.
4. Your `username`.
5. Personal access token that can be used for authentication. You can generate it by following the link `https://YOUR_GETI_HOST/account/personal-access-token/` -> `Create`.
6. Proxies environment variable set up on your machine if needed.


In [2]:
HOST = "https://geti-demo.iotg.sclab.intel.com/"
USERNAME = "=="
TOKEN = "=="

# Following the information from the API documentation we append the version api version to our host url.
# All the endpoints include the API version in their address and will rely on this variable.
HOST = HOST + "api/v1/"

# We will include the `x-api-key` header to all our API calls, this way the server will be able to authenticate our requests.
headers = {"x-api-key": TOKEN}

In [3]:
# Certification?

# import certifi
# certifi.where()

## Step 1: Obtain the Organization and Workspace IDs
In Intel Geti projects belong to workspaces, workspaces in their turn belong to organizations.
To interact with a project, we need to determine the `organization_id` and `workspace_id` that we want to use. A `GET` request to the `personal_access_tokens/organization` endpoint will get us a response containing the ID of the organization that you have access to.

In [4]:
import requests

# Disable certification warnings
import urllib3

urllib3.disable_warnings()

response = requests.get(
    HOST + "personal_access_tokens/organization", headers=headers, verify=False
)
print(response.json())

# Save the organization ID for future use
ORG_ID = response.json()["organizationId"]

{'organizationId': 'c16095ac-60bf-4931-b059-c83f508308ac'}


 After that, you can call the `GET /organizations/{organization_id}/workspaces/` endpoint to get the list of workspaces in the organization. Choose the workspace you want to use and get the `workspace_id` from the response.

In [5]:
response = requests.get(
    HOST + f"organizations/{ORG_ID}/workspaces", headers=headers, verify=False
).json()

WORKSPACE_ID = response["workspaces"][0]["id"]

response

{'workspaces': [{'id': '92597063-e623-42ea-9e41-e9edb20c14ec',
   'name': 'Default workspace',
   'organizationId': 'c16095ac-60bf-4931-b059-c83f508308ac',
   'createdAt': '2024-06-17T20:23:44Z',
   'createdBy': '',
   'modifiedAt': None,
   'modifiedBy': ''}],
 'totalCount': 1,
 'totalMatchedCount': 1,
 'nextPage': {'skip': 0, 'limit': 0}}

Let's update the base URL, adding the `organization_id`, and `workspace_id` to the HOST name, it will shorten the URL for the following requests.

In [6]:
HOST = HOST + "organizations/" + ORG_ID + "/" + "workspaces/" + WORKSPACE_ID + "/"

print(HOST)

https://geti-demo.iotg.sclab.intel.com/api/v1/organizations/c16095ac-60bf-4931-b059-c83f508308ac/workspaces/92597063-e623-42ea-9e41-e9edb20c14ec/


## Create a project
Let's create a simple detection project. The REST API reference suggests that we need to send a POST request to the `/organizations/{organization_id}/workspaces/{workspace_id}/projects` endpoint. The endpoint description also contains the message schema, lists available project types and constrains on the label configuration in the project.
The request body should contain the project name and pipeline configuration.
Pipeline configuration is a JSON object that describes the task execution graph, is contains a list of tasks in the Project, labels for those tasks, and the connections between the tasks.
Although it is possible to create longer task-chain projects with Intel Geti, in this tutorial we will create a simple project with just two tasks.
1. The first task has the `dataset` type and all the projects In Intel Geti start with this tasks. It represents the training dataset retrieval and preparation for the model training.
>Note The only other member of the  utility task family is the `crop` task, which you put between trainable tasks in chained projects to crop the output of the previous task before passing it to the next task. Detection -> Crop -> Classification, for example. The `crop` task is not used in this tutorial.
2. The second task is the `detection` type task. This task have a trainable model behind it, see the full list of currently supported trainable tasks:
    * classification
    * detection
    * rotated_detection
    * segmentation
    * instance_segmentation
    * anomaly_classification
    * anomaly_detection
    * anomaly_segmentation

For the detection task we will also create an `object` label, which will be used to annotate the image. The documentation says: "A default 'No object' label is automatically created for detection task", so we don't need to create it manually.

Finally, the `connections` is just a list of dictionaries, where each dictionary contains the `from` and `to` keys pointing to the titels of the tasks in the pipeline configuration list. In our case, the project graph contains only two tasks, so we need only one connection.

In [7]:
project_name = "TUTORIAL DETECTION PROJECT"
tasks = [
    {
        "task_type": "dataset",
        "title": "Dataset",
    },
    {
        "task_type": "detection",
        "title": "Detection Task",
        "labels": [
            {
                "name": "object",
                "color": "#ff0000",
                "group": "default_detection",
            },
        ],
    },
]
connections = [
    {
        "from": "Dataset",
        "to": "Detection Task",
    },
]

For longer task chaines, if you listed the tasks in the logical order, you can use a snippet like this to create the connections:
```python
connections = [
    {
        "from": prev_task["title"],
        "to": next_task["title"],
    } for prev_task, next_task in zip(tasks[:-1], tasks[1:])
]
```

### Compile the message and fire the Project Creation request
Let's just pack all the message parts in to the request body and send it to the server.

In [8]:
project_creation_request_data = {
    "name": project_name,
    "pipeline": {
        "connections": connections,
        "tasks": tasks,
    },
}

In [9]:
response = requests.post(
    HOST + "projects", headers=headers, json=project_creation_request_data, verify=False
)
project_dict = response.json()

project_dict

{'id': '667a68ce4994eaba0842f320',
 'name': 'TUTORIAL DETECTION PROJECT',
 'creation_time': '2024-06-25T06:50:54.634000+00:00',
 'creator_id': 'fa772443-859e-4c57-9b12-294fff537bb3',
 'pipeline': {'tasks': [{'id': '667a68ce4994eaba0842f321',
    'title': 'Dataset',
    'task_type': 'dataset'},
   {'id': '667a68ce4994eaba0842f323',
    'title': 'Detection Task',
    'task_type': 'detection',
    'labels': [{'id': '667a68ce4994eaba0842f325',
      'name': 'object',
      'is_anomalous': False,
      'color': '#ff0000ff',
      'hotkey': '',
      'is_empty': False,
      'group': 'default_detection',
      'parent_id': None},
     {'id': '667a68ce4994eaba0842f329',
      'name': 'No object',
      'is_anomalous': False,
      'color': '#000000ff',
      'hotkey': '',
      'is_empty': True,
      'group': 'No object',
      'parent_id': None}],
    'label_schema_id': '667a68ce4994eaba0842f32b'}],
  'connections': [{'from': '667a68ce4994eaba0842f321',
    'to': '667a68ce4994eaba0842f323'}

Take a look at the response the server sends back. The server assigned `IDs` to all the entities in the project, including the project itself, tasks, and labels. We will need these IDs to modify tasks configuration, assign labels to media items and manage model deployments among other things.

Also note that the server created a default training dataset for us. We will use it to upload an image in the next step.

## Upload an image
Now we will upload a media item to our newly created project. The endpoint for this is \
`/organizations/{organization_id}/workspaces/{workspace_id}/projects/{project_id}/datasets/{dataset_id}/media/images`.\
The request body should contain just the media item bytes, the server will assight it an id on reception.

In [10]:
project_id = project_dict["id"]
dataset_id = project_dict["datasets"][0]["id"]
url_suffix = f"projects/{project_id}/datasets/{dataset_id}/media/images"

In [11]:
# Let's create the image bytes reader and pass it to the requests.post method
file_path = "./cat_sample.jpg"
buffer = open(file_path, "rb")

In [12]:
response = requests.post(
    HOST + url_suffix, headers=headers, files={"file": buffer}, verify=False
)
image_dict = response.json()

image_dict

{'id': '667a68cf4994eaba0842f32c',
 'name': 'cat_sample',
 'type': 'image',
 'media_information': {'height': 426,
  'width': 640,
  'display_url': '/api/v1/organizations/c16095ac-60bf-4931-b059-c83f508308ac/workspaces/92597063-e623-42ea-9e41-e9edb20c14ec/projects/667a68ce4994eaba0842f320/datasets/667a68ce4994eaba0842f326/media/images/667a68cf4994eaba0842f32c/display/full',
  'size': 82118},
 'annotation_state_per_task': [{'task_id': '667a68ce4994eaba0842f323',
   'state': 'none'}],
 'upload_time': '2024-06-25T06:50:55.180000+00:00',
 'uploader_id': 'fa772443-859e-4c57-9b12-294fff537bb3',
 'thumbnail': '/api/v1/organizations/c16095ac-60bf-4931-b059-c83f508308ac/workspaces/92597063-e623-42ea-9e41-e9edb20c14ec/projects/667a68ce4994eaba0842f320/datasets/667a68ce4994eaba0842f326/media/images/667a68cf4994eaba0842f32c/display/thumb'}

Again, take a look at the response. The server assigned an `ID` to the media item, which we will need to annotate it in the next step. It also returned the media item metadata, including the `width` and `height` of the image, its size amd upload time.

### Annotate the image
Finally, we can annotate an object on the image. The endpoint for this is \
`/organizations/{organization_id}/workspaces/{workspace_id}/projects/{project_id}/datasets/{dataset_id}/media/images/{image_id}/annotations`.

In [13]:
image_id = image_dict["id"]

url_suffix = (
    f"projects/{project_id}/datasets/{dataset_id}/media/images/{image_id}/annotations"
)

The request body for this endpoint should contain a list of annotations. Each annotation is a dictionary with the shape of the object you want to annotate and label you want to assign. Documentation contains details on the annotation format for each task type, in our case, we need to provide the `RECTANGLE` type shape with left top `x` and `y` coordinates, and the `width` and `height` of the rectangle. The label should be the `ID` of the label we want to assign to the object.

In [14]:
object_label_id = project_dict["pipeline"]["tasks"][1]["labels"][0]["id"]

annotatiion_request_data = {
    "annotations": [
        {
            "shape": {
                "type": "RECTANGLE",
                "x": 340,
                "y": 153,
                "width": 77,
                "height": 145,
            },
            "labels": [
                {
                    "id": object_label_id,
                }
            ],
        },
    ],
}

In [15]:
response = requests.post(
    HOST + url_suffix, headers=headers, json=annotatiion_request_data, verify=False
)
response.json()

{'kind': 'annotation',
 'modified': '2024-06-25T06:50:55.744000+00:00',
 'media_identifier': {'type': 'image', 'image_id': '667a68cf4994eaba0842f32c'},
 'id': '667a68cf4994eaba0842f32e',
 'labels_to_revisit_full_scene': [],
 'annotation_state_per_task': [{'task_id': '667a68ce4994eaba0842f323',
   'state': 'annotated'}],
 'annotations': [{'id': '667a68cf4994eaba0842f32d',
   'modified': '2024-06-25T06:50:55.744000+00:00',
   'labels': [{'id': '667a68ce4994eaba0842f325',
     'probability': 1.0,
     'source': {'user_id': 'fa772443-859e-4c57-9b12-294fff537bb3',
      'model_id': None,
      'model_storage_id': None}}],
   'labels_to_revisit': [],
   'shape': {'type': 'RECTANGLE',
    'x': 340,
    'y': 153,
    'width': 77,
    'height': 145}}]}

The server will assign an `ID` to each annotation on reception, inspect the response to get the annotation `ID`.

At this point you can log in to the Intel Geti Platform Web UI and see the project you created, the media item you uploaded, and the annotation you made.\
We will look at more complex actions like training and model downloading in the next sections.